In [1]:
import math
import numpy as np
import mpmath

# Problema 2

Pentru $x$ numar mare, eroarea creste foarte mult, si aproximarea nu mai este buna.

$$
\begin{aligned}
\sin{x}&=x-\frac{x^3}{3!}+\frac{x^5}{5!}-\frac{x^7}{7!}-\dots \\
t_n&=(-1)^{n}\frac{x^{2n+1}}{(2n+1)!} \\
t_{n+1}&=(-1)\frac{x^2}{(2n+2)(2n+3)}t_n \\
\end{aligned}
$$

In [2]:
def taylor_sin_basic(x):
    x = float(x)

    r = 0
    t = x
    n = 1
    while True:
        r += t
        
        s = -1 * ((x*x) / ((2*n)*(2*n+1))) * t
        if s == t:
            break
        t = s
        n += 1
    
    return r


def print_sin(x):
    print(math.sin(x))
    print(taylor_sin_basic(x))


print_sin(0)
print_sin(math.pi*10)
print_sin(math.pi*(1/2))

0.0
0.0
-1.2246467991473533e-15
-0.0003048974510408005
1.0
1.0000000000000002


$$
\begin{aligned}
\cos{x}&=1-\frac{x^2}{2!}+\frac{x^4}{4!}-\frac{x^6}{6!}+\dots \\
t_n&=(-1)^{n}\frac{x^{2n}}{(2n)!} \\
t_{n+1}&=(-1)\frac{x^2}{(2n+1)(2n+2)}t_n \\
\end{aligned}
$$

In [3]:
def taylor_cos_basic(x):
    x = float(x)

    r = 0
    t = 1
    n = 1
    while True:
        r += t
        
        s = -1 * ((x*x) / ((2*n-1)*(2*n))) * t
        if s == t:
            break
        t = s
        n += 1
    
    return r


def print_cos(x):
    print(math.cos(x))
    print(taylor_cos_basic(x))


print_cos(0)
print_cos(math.pi*10)
print_cos(math.pi*(1/2))

1.0
1.0
1.0
1.0001203373541816
6.123233995736766e-17
4.5907218229751464e-17


---

Solutie propusa: facem ca $x$ sa fie intotdeauna in intervalul $[0,2\pi)$.

$$x'=x-\left[\frac{x}{2\pi}\right]\cdot2\pi$$

In [47]:
def taylor_sin_better(x):
    x = float(x)
    x -= math.floor(x / math.tau) * math.tau
    return taylor_sin_basic(x)


def print_sin(x):
    print(math.sin(x))
    print(taylor_sin_better(x))


print_sin(0)
print_sin(math.pi*10)
print_sin(math.pi*(10**15 + .25))

0.0
0.0
-1.2246467991473533e-15
0.0
0.6900349619879262
0.8414709848078965


In [48]:
def taylor_cos_better(x):
    x = float(x)
    x -= math.floor(x / math.tau) * math.tau
    return taylor_cos_basic(x)


def print_cos(x):
    print(math.cos(x))
    print(taylor_cos_better(x))


print_cos(0)
print_cos(math.pi*10)

1.0
1.0
1.0
1.0


---

Solutia cea mai buna: reducere la primul cadran cu 100% precizie.

In [59]:
@mpmath.workprec(1144)
def taylor_sin_best(x):
    x = mpmath.mpf(x)
    y = x * (2 / mpmath.pi)
    k = mpmath.floor(y)
    f = y - k
    r = f * (mpmath.pi / 2)

    r = float(r)
    k = int(k) % 4
    match k:
        case 0:
            return taylor_sin_basic(r)
        case 1:
            return taylor_cos_basic(r)
        case 2:
            return -taylor_sin_basic(r)
        case 3:
            return -taylor_cos_basic(r)
        case _:
            assert False


print(math.sin(math.pi * .25))

v = math.pi*(10**10 + .25)
print(taylor_sin_better(v))
print(taylor_sin_best(v))

0.7071067811865475
0.7071083560724986
0.7071067726056579


In [61]:
@mpmath.workprec(1144)
def taylor_cos_best(x):
    x = mpmath.mpf(x)
    y = x * (2 / mpmath.pi)
    k = mpmath.floor(y)
    f = y - k
    r = f * (mpmath.pi / 2)

    r = float(r)
    k = int(k) % 4
    match k:
        case 0:
            return taylor_cos_basic(r)
        case 1:
            return -taylor_sin_basic(r)
        case 2:
            return -taylor_cos_basic(r)
        case 3:
            return taylor_sin_basic(r)
        case _:
            assert False


print(math.cos(math.pi * .25))

v = math.pi*(10**10 + .25)
print(taylor_cos_better(v))
print(taylor_cos_best(v))

0.7071067811865476
0.7071052062970886
0.7071067897674369


# Problema 3

In [ ]:
def make_pade(d_fn):
    def fn(x, k, m):
        def c(i):
            if i < 0:
                return 0
            return d_fn(i) / math.factorial(i)
    
        mat = np.array([
            [
                c(m+i-j)
                for j in range(k)
            ]
            for i in range(k)
        ])
        vec = np.array([
            -c(m+i+1)
            for i in range(k)
        ])
        b = np.linalg.solve(mat, vec)
        b = np.concatenate([np.array([1]), b])
        
        a = []
        for j in range(m+1):
            ac = 0
            for l in range(j+1):
                ac += c(j-l)*b[l]
            a.append(ac)
        a = np.array(a)
    
        sol = (
            np.sum(a * (x**np.arange(m+1)))
            /
            np.sum(b * (x**np.arange(k+1)))
        )
        return sol
    return fn

In [ ]:
def sin_d_0(i):
    match i%4:
        case 0:
            return 0
        case 1:
            return 1
        case 2:
            return 0
        case 3:
            return -1
        case _:
            assert False


pade_sin = make_pade(sin_d_0)


print(pade_sin(0, 2, 2))
print(pade_sin(math.pi, 2, 2))
print(pade_sin(math.pi, 10, 5))
print(pade_sin(math.pi*10, 2, 2))
print(pade_sin(math.pi*10, 10, 5))

In [ ]:
def cos_d_0(i):
    match i%4:
        case 0:
            return 1
        case 1:
            return 0
        case 2:
            return -1
        case 3:
            return 0
        case _:
            assert False


pade_cos = make_pade(cos_d_0)


print(pade_cos(0, 2, 2))
print(pade_cos(math.pi, 2, 2))
print(pade_cos(math.pi, 10, 5))
print(pade_cos(math.pi*10, 2, 2))
print(pade_cos(math.pi*10, 10, 5))